In [1]:
import os
import numpy as np
from sklearn import svm

In [2]:
#reading of the emails (enron)
emailspam=[]
directory="data/spam"
for file in sorted(os.listdir(directory)):
    f=open(directory+"/"+file,"r",encoding='cp437')
    b=f.readline().lower()
    b=b.replace("subject: fw :","")
    b=b.replace("subject: fw : fw :","")
    b=b.replace("subject: re : re :","")
    b=b.replace("subject: re :","")
    b=b.replace("subject: ","")
    a=b+f.read().lower()
    emailspam.append(a)
    
emailham=[]
directory="data/ham"
for file in sorted(os.listdir(directory)):
    f=open(directory+"/"+file,"r",encoding='cp437')
    a=f.readline().lower()
    start=1
    b=""
    while a!="":
        if start==1:
            b+=a
            a=f.readline()
        else:
            a=f.readline()
            
        if a[:7]=="subject":
            start=1
        elif a[:10]=="- - - - - " or a=="":
            start=0
            emailham.append(b)
            b=""

            
for i in range(len(emailham)):
    b=emailham[i]
    b=b.replace("subject: fw :","")
    b=b.replace("subject: fw : fw :","")
    b=b.replace("subject: re : re :","")
    b=b.replace("subject: re :","")
    b=b.replace("subject: ","")
    emailham[i]=b
    
s_len=len(emailspam)
h_len=len(emailham)
emails=emailspam+emailham
labels=np.hstack(((np.ones(s_len),np.zeros(h_len))))
emails=np.vstack((np.array(emails).transpose(),labels)).transpose()          

In [4]:
def symbol_color_removal(email):
    e_mod=email
    sym=["\\","&","!",".",",","/","+","*","%","(",")",":",";","\"","_","?","=","@","{","}","[","]","<",">","|"]
    f=open("data/color_codes.txt","r")
    color=f.read().split("\n")
    for s in sym:
        e_mod=e_mod.replace(s," ")
    e_mod=e_mod.replace("\'","")
    e_mod=e_mod.replace("\n"," ")
    e_mod=e_mod.replace("-","")
    for c in color:
        e_mod=e_mod.replace(c,"")

    return e_mod

def lemmatisation(email,dic):
    
    for key in dic.keys():
        e_mod=email.replace(key," "+dic[key]+" ")
    
    return e_mod


def num_token(email):
    num=["0","1","2","3","4","5","6","7","8","9"]
    e_mod=email
    for n in num:
        e_mod=e_mod.replace(n," ")
    
    return e_mod

def stop_words():
    f=open("data/stop_words.txt","r")
    stop_words=f.read().split("\n")

    return stop_words

def html_words():
    f=open("data/html_words.txt","r")
    h_words=f.read().split("\n")

    return h_words

def rtf_words():
    f=open("data/rtf_words.txt","r")
    r_words=f.read().split("\n")

    return r_words

def idf(bag,lis):
    idf_list=[]
    N=len(lis)
    for term in bag:
        a=0
        for doc in lis:
            if term in doc:
                a+=1
        if a==0:
            a=N
        idf_list.append(a)
    idf_vector=np.log(N/np.array(idf_list,dtype="float64"))
    return idf_vector

def tf(bag,doc):
    N=len(bag)
    tf_list=[]
    for i in range(N):
        count=doc.count(bag[i])
        tf_list.append(count/len(doc))
    tf_vector=np.array(tf_list,dtype="float64")
    return tf_vector

def tf_idf_vectorisation(bag,emails):
    idf_v=idf(bag,emails)
    N=len(bag)
    num_of_docs=len(emails)
    X=np.empty((N,num_of_docs))
    for i in range(num_of_docs):
        tf_v=tf(bag,emails[i])
        X[:,i]=tf_v*idf_v
    
    return X

def smote(X,N,k=5):
    X_mod=X.transpose()
    for i in range(X.shape[1]):
        current=X[:,i]
        dist=np.linalg.norm(X.transpose()-X[:,i],axis=1)
        dist_list=dist.tolist()
        neighbours=[]
        for j in range(k):
            min_pos=dist_list.index(min(dist_list))
            neighbours.append(X[:,min_pos])
            dist_list[min_pos]=max(dist_list)+1000

        a=np.arange(k)
        np.random.shuffle(a)
        neighbours_array=(np.array(neighbours))[a[:N]]
        c=np.random.uniform(0.0001,1,N)
        new_x=current+(c*(neighbours_array-current).transpose()).transpose()
        X_mod=np.vstack((X_mod,new_x))
        
    return (X_mod.transpose())

def split(emails,h_len,s_len):


    emails_spam=emails[:s_len]
    emails_ham=emails[s_len:]
    
    np.random.shuffle(emails_spam)
    np.random.shuffle(emails_ham)
    
    emails_spam_train=emails_spam[:int(0.7*s_len)]
    emails_ham_train=emails_ham[:int(0.7*h_len)]
    
    emails_spam_test=emails_spam[int(0.7*s_len):]
    emails_ham_test=emails_ham[int(0.7*h_len):]
    
    emails_train=emails_spam_train+emails_ham_train
    emails_test=emails_spam_test+emails_ham_test
    label_train=np.hstack((np.ones(len(emails_spam_train)),np.zeros(len(emails_ham_train))))
    label_test=np.hstack((np.ones(len(emails_spam_test)),np.zeros(len(emails_ham_test))))
    
    return emails_train,emails_test,label_train,label_test

In [6]:
lemma_dictionary=np.load("lemma_dictionary.npy",allow_pickle=True).tolist()
for i in range(len(emails)):
    emails[i,0]=symbol_color_removal(emails[i,0])
    emails[i,0]=num_token(emails[i,0])
    emails[i,0]=lemmatisation(emails[i,0],lemma_dictionary)
    
s_words=stop_words()
h_words=html_words()
r_words=rtf_words()
length=len(emails)
emails_list=[]
emails_list_labels=[]
for i in range(length):
    e=emails[i,0].strip().split(" ")
    l=[x for x in e if x not in s_words and  x not in h_words and x not in r_words and x!=" " and x!="" and x!="#"]
    if l!=[]:
        emails_list.append(l)
        emails_list_labels.append(emails[i,1])

spam_len=list(map(float,emails_list_labels)).index(0.0)
ham_len=len(emails_list_labels)-spam_len

email_train,email_test,label_train,label_test=split(emails_list,ham_len,spam_len)

emails_train_flat=[i for j in email_train for i in j]
emails_bag=list(set(emails_train_flat))


X_train_nolabel=tf_idf_vectorisation(emails_bag,email_train)

X_test=tf_idf_vectorisation(emails_bag,email_test).transpose()

no_of_spams=(label_train.tolist()).index(0.0)
no_of_hams=len(label_train)-no_of_spams
rate=int(no_of_hams/no_of_spams) -1

spam_balanced=smote(X_train_nolabel[:,:no_of_spams],rate)

X_train=np.vstack((spam_balanced.transpose(),X_train_nolabel[:,no_of_spams:].transpose()))
spam_count_balanced=spam_balanced.shape[1]
y_train=np.hstack((np.ones(spam_count_balanced),np.zeros(no_of_hams)))

lsvc=svm.LinearSVC(C=10,max_iter=10000)
lsvc.fit(X_train,y_train)
y_pred=lsvc.predict(X_test)
accuracy=np.sum(1-np.abs(y_pred-label_test))/len(label_test)
print(accuracy)